In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

###merging 12 months of sales data into a single file

In [ ]:
files=[file for file in os.listdir('/kaggle/input/sales-data/Pandas-Data-Science-Tasks-master/SalesAnalysis/Sales_Data')]
all_months_data=pd.DataFrame()
for file in files:
    df=pd.read_csv("/kaggle/input/sales-data/Pandas-Data-Science-Tasks-master/SalesAnalysis/Sales_Data/"+file)
    all_months_data=pd.concat([all_months_data,df])
all_months_data.to_csv('all_data.csv',index=False)

In [ ]:
all_data=pd.read_csv('/kaggle/working/all_data.csv')
all_data

In [ ]:
all_data.head()

In [ ]:
all_data.info()

**CLEAN UP THE DATA**

drop rows of nan

In [ ]:
all_data=all_data.dropna(how='all')
all_data.head()

In [ ]:
all_data.info()

Find 'Or' and delete it--
error was coming while converting month into int

In [ ]:
all_data=all_data[all_data['Order Date'].str[0:2]!='Or']

In [ ]:
all_data.head()

**### Convert columns to the correct type**
as beacuse to make sales column

In [ ]:
all_data['Quantity Ordered']=pd.to_numeric(all_data['Quantity Ordered'])
all_data['Price Each']=pd.to_numeric(all_data['Price Each'])

In [ ]:
all_data.head()

**additional column adding**

**adding month column**

In [ ]:
all_data['Month']=all_data['Order Date'].str[0:2]
all_data['Month']=all_data['Month'].astype('int32')
all_data

**adding a sales column**

In [ ]:
all_data['Sales']=all_data['Quantity Ordered']*all_data['Price Each']
all_data.head()

**Q:1-->  What was the best month for sales?How much was earned that month**

In [ ]:
all_data.groupby('Month').sum()

In [ ]:
all_data.groupby('Month').sum().Sales.sort_values(ascending=False)

In [ ]:
results=all_data.groupby('Month').sum()
months=range(1,13)
plt.bar(months,results['Sales'])

plt.show()

Q-2--> What city had the highest number of sales??

**Add a city column**

In [ ]:
all_data.head()

In [ ]:
### Lets use the .apply()
##all_data['City']=all_data['Purchase Address'].apply(lambda x:x.split(',')[1])
##all_data.head()
## OR we can make a function
def get_city(address):
   return address.split(',')[1]
    
## due to same cities across us in different parts 
## we need to grap some more information like address using above same function

def get_state(address):
    return address.split(',')[2].split(' ')[1]## additional split used to remove zip code/pin code

all_data['City']=all_data['Purchase Address'].apply(lambda x:get_city(x)+' '+get_state(x))


In [ ]:
all_data.City

In [ ]:
results=all_data.groupby('City').sum()['Sales'].sort_values(ascending=False).reset_index(name='Total_Sales_in_City')
results

In [ ]:
plt.bar(results.City,results.Total_Sales_in_City)
plt.xticks(results.City,rotation='vertical',size=10)
plt.show()

**Q-3---> What time should we display advertisements to maximize likelihood of customers buying product?**

In [ ]:
all_data.head()['Order Date']

In [ ]:
all_data['Order Date']=pd.to_datetime(all_data['Order Date'])

now you can see the date time format has been changed

In [ ]:
all_data.head()

In [ ]:
all_data['Hour']=all_data['Order Date'].dt.hour
all_data['Minute']=all_data['Order Date'].dt.minute
all_data.head()

In [ ]:
results=all_data.groupby('Hour').count()['Sales'].reset_index(name='Count')
results

In [ ]:
plt.plot(results.Hour,results.Count)
plt.xticks(results.Hour)
plt.grid()
plt.xlabel('Hour')
plt.ylabel('Number of Orders')
plt.show

above graph is drawn by simple my understanding and below graph is drawn by tutorial

In [ ]:
hours=[hour for hour,df in all_data.groupby('Hour')]
plt.plot(hours,all_data.groupby(['Hour']).count())
plt.xticks(hours)
plt.grid()
plt.xlabel('Hour')
plt.ylabel('Number of Orders')
plt.show()

**Q-4-->  What products are most often sold together?**

In [ ]:
all_data.head()

In [ ]:
df=all_data[all_data['Order ID'].duplicated(keep=False)]
df.head(20)

In [ ]:
df['Grouped']=df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
df.head(5)

In [ ]:
df=df[['Order ID','Grouped']].drop_duplicates()
df.head(10)

In [ ]:
from itertools import combinations
from collections import Counter

count=Counter()

for row in df['Grouped']:
    row_list=row.split(',')
    count.update(Counter(combinations(row_list,2)))

for key,value in count.most_common(10):
    print(key,value)

**Q-5-->  What product sold the most?Why do you think it sold the most?**

In [ ]:
all_data.head()

In [ ]:
new_df=all_data[['Product','Quantity Ordered']]
new_df

In [ ]:
product_group=new_df.groupby('Product')
quantity_ordered=product_group.sum()['Quantity Ordered']


In [ ]:
products=[product for product,df in product_group]
plt.bar(products,quantity_ordered)
plt.xticks(products,rotation='vertical',size=10)
plt.ylabel('Quantity Ordered')
plt.xlabel('Product')
plt.show()